In [ ]:
import os
import base64
import google.generativeai as genai
import requests  # For Slack integration
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Load environment variables from .env file
load_dotenv()

# Fetch API keys from .env
GEMINI_API_KEY = os.getenv("gemini_api_key")
SLACK_WEBHOOK_URL = os.getenv("slack_webhook_url")  # Get Slack Webhook URL

# Configure Gemini AI
genai.configure(api_key=GEMINI_API_KEY)

# Gmail API Scopes (Read-only)
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# Define important keywords for filtering
IMPORTANT_KEYWORDS = [
    "job", "interview", "hiring", "career", "HR", "offer letter",
    "appointment", "meeting", "schedule", "calendar invite",
    "urgent", "emergency", "asap", "critical"
]

def get_gmail_service():
    """Authenticate and get the Gmail API service."""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

def fetch_important_emails():
    """Fetch unread emails and filter only important ones."""
    service = get_gmail_service()
    
    # Get unread emails
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread").execute()
    messages = results.get('messages', [])

    important_emails = []

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
        
        # Extract email snippet and sender details
        snippet = msg_data.get("snippet", "")
        payload = msg_data.get("payload", {})
        body = ""
        sender = ""
        
        # Extract sender's email ID
        headers = payload.get("headers", [])
        for header in headers:
            if header['name'] == "From":
                sender = header['value']

        # Extract email body
        if "parts" in payload:
            for part in payload.get("parts", []):
                if part.get("mimeType") == "text/plain":
                    body_data = part.get("body", {}).get("data", "")
                    if body_data:
                        body = base64.urlsafe_b64decode(body_data).decode("utf-8")

        # Check if email contains any important keyword
        email_text = snippet.lower() + " " + body.lower()
        if any(keyword in email_text for keyword in IMPORTANT_KEYWORDS):
            important_emails.append({'id': msg['id'], 'sender': sender, 'snippet': snippet, 'body': body})

    return important_emails

def summarize_email_gemini(email_body):
    """Summarize the email using Gemini AI."""
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(f"Summarize this email: {email_body}")
    return response.text  # Get summary text

def send_slack_notification(sender, summary, snippet):
    """Send an alert to Slack with the important email details."""
    message = f"""
    🚨 *New Important Email Alert!* 🚨
    📧 *Sender:* {sender}
    ✍️ *Summary:* {summary}
    📌 *Snippet:* {snippet}
    """
    
    payload = {"text": message}
    response = requests.post(SLACK_WEBHOOK_URL, json=payload)

    if response.status_code == 200:
        print("✅ Slack notification sent successfully!")
    else:
        print(f"❌ Failed to send Slack notification. Error: {response.text}")

# Fetch and summarize important emails
important_emails = fetch_important_emails()

if important_emails:
    for email in important_emails:
        summary = summarize_email_gemini(email["body"])
        print("\n🔹 **Important Email Found** 🔹")
        print(f"📧 Sender: {email['sender']}")
        print(f"✍️ Summary: {summary}")
        print(f"📌 Snippet: {email['snippet']}")
        print("="*50)

        # Send Slack notification
        send_slack_notification(email["sender"], summary, email["snippet"])
else:
    print("✅ No important emails found.")
